<a href="https://colab.research.google.com/github/s34836/EWD/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

--2025-03-12 13:16:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘iris.data’

iris.data               [ <=>                ]   4.44K  --.-KB/s    in 0s      

2025-03-12 13:16:57 (62.3 MB/s) - ‘iris.data’ saved [4551]



In [2]:
import pandas as pd

iris_raw = pd.read_csv('iris.data', header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])
iris = iris_raw.copy()
iris.head()


,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   class         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [4]:
iris.describe().T

,count,mean,std,min,25%,50%,75%,max
sepal_length,150.0,5.843333,0.828066,4.3,5.1,5.80,6.4,7.9
sepal_width,150.0,3.054000,0.433594,2.0,2.8,3.00,3.3,4.4
petal_length,150.0,3.758667,1.764420,1.0,1.6,4.35,5.1,6.9
petal_width,150.0,1.198667,0.763161,0.1,0.3,1.30,1.8,2.5


In [5]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.7 MB/s eta 0:00:00


In [6]:


import openmeteo_requests
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


# WARSAW 52.237049, 21.017532
def download_weather_data(latitude = 52.237049, longitude = 21.017532, start_date='2024-01-01', end_date='2024-12-31'):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ["temperature_2m", "rain", "cloud_cover", "apparent_temperature", "precipitation", "snowfall", "pressure_msl"]
    }

    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    data = response.Hourly()
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_rain = hourly.Variables(1).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(6).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
      start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
      end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
      freq = pd.Timedelta(seconds = hourly.Interval()),
      inclusive = "left"
    )}

    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["rain"] = hourly_rain
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["pressure_msl"] = hourly_pressure_msl

    return pd.DataFrame(data = hourly_data)

weather_raw= download_weather_data()
weather = weather_raw.copy()
weather.set_index('date', inplace=True)
weather.head()


,temperature_2m,rain,cloud_cover,apparent_temperature,precipitation,snowfall,pressure_msl
date,,,,,,,
2024-01-01 00:00:00+00:00,3.0375,0.1,100.0,-0.114680,0.1,0.0,1008.700012
2024-01-01 01:00:00+00:00,2.2875,0.0,100.0,-1.029752,0.0,0.0,1008.200012
2024-01-01 02:00:00+00:00,2.0375,0.0,100.0,-1.010007,0.0,0.0,1007.799988
2024-01-01 03:00:00+00:00,2.5875,0.0,100.0,-0.140697,0.0,0.0,1007.599976
2024-01-01 04:00:00+00:00,2.7375,0.0,100.0,-0.081075,0.0,0.0,1007.000000


In [7]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8784 entries, 2024-01-01 00:00:00+00:00 to 2024-12-31 23:00:00+00:00
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   temperature_2m        8784 non-null   float32
 1   rain                  8784 non-null   float32
 2   cloud_cover           8784 non-null   float32
 3   apparent_temperature  8784 non-null   float32
 4   precipitation         8784 non-null   float32
 5   snowfall              8784 non-null   float32
 6   pressure_msl          8784 non-null   float32
dtypes: float32(7)
memory usage: 308.8 KB


In [8]:
!pip install geopy

In [9]:
from geopy.geocoders import Nominatim
from pprint import pprint

app = Nominatim(user_agent="PJWSTK")
location = app.geocode('Polsko Japońska Akademia Technik Komputerowych').raw

weather_raw_PJA= download_weather_data(float(location['lat']), float(location['lon']))
weather_PJA = weather_raw_PJA.copy()
weather_PJA.set_index('date', inplace=True)
weather_PJA.head()


,temperature_2m,rain,cloud_cover,apparent_temperature,precipitation,snowfall,pressure_msl
date,,,,,,,
2024-01-01 00:00:00+00:00,3.257,0.1,100.0,0.122712,0.1,0.0,1008.599976
2024-01-01 01:00:00+00:00,2.307,0.0,100.0,-1.066256,0.0,0.0,1008.099976
2024-01-01 02:00:00+00:00,2.307,0.0,100.0,-0.735433,0.0,0.0,1007.799988
2024-01-01 03:00:00+00:00,2.807,0.0,100.0,0.145427,0.0,0.0,1007.599976
2024-01-01 04:00:00+00:00,2.857,0.0,100.0,0.061179,0.0,0.0,1007.000000


In [10]:
weather_PJA.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8784 entries, 2024-01-01 00:00:00+00:00 to 2024-12-31 23:00:00+00:00
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   temperature_2m        8784 non-null   float32
 1   rain                  8784 non-null   float32
 2   cloud_cover           8784 non-null   float32
 3   apparent_temperature  8784 non-null   float32
 4   precipitation         8784 non-null   float32
 5   snowfall              8784 non-null   float32
 6   pressure_msl          8784 non-null   float32
dtypes: float32(7)
memory usage: 308.8 KB


In [11]:
import requests

def get_top_articles(year, month):

  base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/top"
  url = f"{base_url}/en.wikipedia/all-access/{year}/{month:02d}/all-days"
  #print(url)
  headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}


  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes
    data = response.json()
    articles = data['items'][0]['articles']
    return pd.DataFrame(articles[:15]) # Return the top 10 articles
  except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
    return []

# Example usage: get the top 10 articles for January 2022
top_articles_january_2022 = get_top_articles(2022, 1)
top_articles_january_2022 = top_articles_january_2022[top_articles_january_2022.article != 'Main_Page']
top_articles_january_2022 = top_articles_january_2022[top_articles_january_2022.article != 'Special:Search']
top_articles_january_2022 = top_articles_january_2022[:10]
top_articles_january_2022.reset_index(drop=True, inplace=True)
top_articles_january_2022


,article,views,rank
0,Bob_Saget,6807825,3
1,Betty_White,5900462,4
2,Bible,4488269,5
3,Spider-Man:_No_Way_Home,4396131,6
4,Cleopatra,4131379,7
5,Meat_Loaf,3956170,8
6,Deaths_in_2022,3926077,9
7,Eternals_(film),3552382,10
8,Pushpa:_The_Rise,3482817,11
9,Euphoria_(American_TV_series),3276084,12


In [12]:

def get_article_views(name, year):
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/"
    url = f"{base_url}{name}/daily/{year}0101/{year}1231"  # Construct URL for entire year
    #print(url)
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        # Extract total views for the year
        total_views = 0
        if 'items' in data:
            for item in data['items']:
                total_views += item['views']
        return total_views
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {name} in {year}: {e}")
        return None

top_articles_january_2022['total_2022'] = top_articles_january_2022['article'].apply(lambda x: get_article_views(x, 2022))
top_articles_january_2022

,article,views,rank,total_2022
0,Bob_Saget,6807825,3,9322612
1,Betty_White,5900462,4,8318810
2,Bible,4488269,5,24939099
3,Spider-Man:_No_Way_Home,4396131,6,13797492
4,Cleopatra,4131379,7,56372902
5,Meat_Loaf,3956170,8,5852989
6,Deaths_in_2022,3926077,9,49626203
7,Eternals_(film),3552382,10,7534484
8,Pushpa:_The_Rise,3482817,11,7683786
9,Euphoria_(American_TV_series),3276084,12,12465371


In [13]:
census_raw = pd.read_csv('censusData-with3errors')
census = census_raw.copy()
census.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
census.columns = [col.replace('-', '_') for col in census.columns]
census.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [15]:
census.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [16]:
census.describe().T

,count,mean,std,min,25%,50%,75%,max
age,32561.0,38.586745,13.686097,3.0,28.0,37.0,48.0,237.0
fnlwgt,32561.0,189778.366512,105549.977697,12285.0,117827.0,178356.0,237051.0,1484705.0
education_num,32561.0,10.080679,2.572720,1.0,9.0,10.0,12.0,16.0
capital_gain,32561.0,1077.648844,7385.292085,0.0,0.0,0.0,0.0,99999.0
capital_loss,32561.0,87.303830,402.960219,0.0,0.0,0.0,0.0,4356.0
hours_per_week,32561.0,40.437456,12.347429,1.0,40.0,40.0,45.0,99.0


In [17]:
census.sort_values(by='age', ascending=False).head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
7731,237,Private,210945,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,24,United-States,<=50K
222,90,Private,51744,HS-grad,9,Never-married,Other-service,Not-in-family,Black,Male,0,2206,40,United-States,<=50K
25303,90,?,175444,7th-8th,4,Separated,?,Not-in-family,White,Female,0,0,15,United-States,<=50K
5272,90,Private,141758,9th,5,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
1935,90,Private,221832,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,<=50K


Error in line 7731 Age == 237

In [18]:
census.sort_values(by='age', ascending=True).head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
7326,3,State-gov,354929,Assoc-acdm,12,Divorced,Protective-serv,Not-in-family,Black,Male,0,0,38,United-States,<=50K
30026,17,Private,160029,11th,7,Never-married,Other-service,Other-relative,White,Female,0,0,22,United-States,<=50K
421,17,Private,175024,11th,7,Never-married,Handlers-cleaners,Own-child,White,Male,2176,0,18,United-States,<=50K
26189,17,Private,230789,9th,5,Never-married,Sales,Own-child,Black,Male,0,0,22,United-States,<=50K
18516,17,Private,194612,11th,7,Never-married,Other-service,Own-child,White,Male,0,0,25,United-States,<=50K


Error in line 7326 Age == 3 and Divorced

In [19]:
census.sort_values(by='hours_per_week', ascending=True).head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
19750,23,Private,72887,HS-grad,9,Never-married,Craft-repair,Own-child,Asian-Pac-Islander,Male,0,0,1,Vietnam,<=50K
25078,74,Private,260669,10th,6,Divorced,Other-service,Not-in-family,White,Female,0,0,1,United-States,<=50K
11451,27,Private,147951,HS-grad,9,Never-married,Machine-op-inspct,Other-relative,White,Male,0,0,1,United-States,<=50K
8447,67,?,244122,Assoc-voc,11,Widowed,?,Not-in-family,White,Female,0,0,1,United-States,<=50K
32525,81,?,120478,Assoc-voc,11,Divorced,?,Unmarried,White,Female,0,0,1,?,<=50K


In [20]:
census.groupby('education').count()

,age,workclass,fnlwgt,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
education,,,,,,,,,,,,,,
10th,933,933,933,933,933,933,933,933,933,933,933,933,933,933
11th,1175,1175,1175,1175,1175,1175,1175,1175,1175,1175,1175,1175,1175,1175
12th,433,433,433,433,433,433,433,433,433,433,433,433,433,433
1st-4th,168,168,168,168,168,168,168,168,168,168,168,168,168,168
5th-6th,333,333,333,333,333,333,333,333,333,333,333,333,333,333
7th-8th,646,646,646,646,646,646,646,646,646,646,646,646,646,646
9th,514,514,514,514,514,514,514,514,514,514,514,514,514,514
Assoc-acdm,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067,1067
Assoc-voc,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382,1382


In [21]:
census.groupby('marital_status').count()

,age,workclass,fnlwgt,education,education_num,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
marital_status,,,,,,,,,,,,,,
Divorced,4443,4443,4443,4443,4443,4443,4443,4443,4443,4443,4443,4443,4443,4443
Married-AF-spouse,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Married-civ-spouse,14976,14976,14976,14976,14976,14976,14976,14976,14976,14976,14976,14976,14976,14976
Married-spouse-absent,418,418,418,418,418,418,418,418,418,418,418,418,418,418
Never-married,10683,10683,10683,10683,10683,10683,10683,10683,10683,10683,10683,10683,10683,10683
Separated,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025,1025
Widowed,993,993,993,993,993,993,993,993,993,993,993,993,993,993


In [22]:
census.groupby('relationship').count()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
relationship,,,,,,,,,,,,,,
Husband,13193,13193,13193,13193,13193,13193,13193,13193,13193,13193,13193,13193,13193,13193
Not-in-family,8305,8305,8305,8305,8305,8305,8305,8305,8305,8305,8305,8305,8305,8305
Other-relative,981,981,981,981,981,981,981,981,981,981,981,981,981,981
Own-child,5068,5068,5068,5068,5068,5068,5068,5068,5068,5068,5068,5068,5068,5068
Unmarried,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446,3446
Wife,1568,1568,1568,1568,1568,1568,1568,1568,1568,1568,1568,1568,1568,1568


In [23]:
census.groupby('native_country').count()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,income
native_country,,,,,,,,,,,,,,
?,583,583,583,583,583,583,583,583,583,583,583,583,583,583
Cambodia,19,19,19,19,19,19,19,19,19,19,19,19,19,19
Canada,121,121,121,121,121,121,121,121,121,121,121,121,121,121
China,75,75,75,75,75,75,75,75,75,75,75,75,75,75
Columbia,59,59,59,59,59,59,59,59,59,59,59,59,59,59
Cuba,95,95,95,95,95,95,95,95,95,95,95,95,95,95
Dominican-Republic,70,70,70,70,70,70,70,70,70,70,70,70,70,70
Ecuador,28,28,28,28,28,28,28,28,28,28,28,28,28,28
El-Salvador,106,106,106,106,106,106,106,106,106,106,106,106,106,106


In [24]:
census[census['native_country'].str.contains("Unimed-States")]

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
32554,53,Private,321865,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,Unimed-States,>50K


Error in line 32554 native-country = 'Unimed-States'